<a href="https://colab.research.google.com/github/IrynaBarnych/AI_EX/blob/main/exam/%D0%95%D0%BA%D0%B7%D0%B0%D0%BC%D0%B5%D0%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Основні умови

Студент обирає **одне завдання** на вибір, або узгоджує власний проект з викладачем. Студент повен показати розуміння роботи з даними та побудови моделі.

Студент самостійно пише увесь код до початку екзамену. Під час екзамену кожен студент презентує свою роботу викладачу та відповідає на основні питання, що стосуються проекту, також можуть бути додаткові питання на кристалики\більші бали.

Робота повинна бути оформлена як готовий код, який на прохання викладача можна буде запустити або внести певні зміни. На вибір студента можна використовувати Colab чи Kaggle

# Завдання 1

Дані про ... Опис даних:
* *case_id:* Ідентифікатор випадку, зареєстрованого в лікарні
* *Hospital_code:* Унікальний код лікарні
* *Hospital_type_code:* Унікальний код типу лікарні
* *City_Code_Hospital:* Код міста, в якому знаходиться Лікарня
* *Hospital_region_code:* Код країни, в якій знаходиться лікарня
* *country_code:* Код області, в якій знаходиться лікарня
* *Available Extra Rooms in Hospital:* Кількість додаткових палат, доступних у лікарні
* *Department:* Відділення, в якому знаходиться пацієнт
* *Ward_Type:* Код типу палати
* *Ward_Facility_Code:* Код приміщення для палати
* *Bed Grade:* Стан ліжка в палаті
* *patientid:* Унікальний ідентифікатор пацієнта
* *doctorid:* Унікальний ідентифікатор лікаря
* *City_Code_Patient:* Код міста для пацієнта
* *Type of Admission:* Тип госпіталізації, зареєстрований лікарнею
* *Severity of Illness:* Тяжкість захворювання, зафіксована на момент госпіталізації
* *Visitors with Patient:* Кількість відвідувачів з пацієнтом
* *Age:* Вік пацієнта
* *Admission_Deposit:* Депозит при госпіталізації
* *Stay:* Дні перебування пацієнта в стаціонарі(цільова ознака)


## Умова

**Тип задачі**

Модель повинна спрогнозувати кількість днів, які пацієнт провів у лікарні. Вам потрібно розв'язати цю задачу як задачу класифікації(спрогнозувати діапазон днів) та задачу регресії(спрогнозувати саму кількість днів), у випадку задачі регресії вам потрібно згенерувати нову цільову ознаку Stay_Days на основі Stay, як середнє значення діапазону.

При тренуванні моделей у вас може бути лише одина ознака Stay_Days чи Stay

**Провести попередній аналіз даних**
* ознайомитись з даними, визначити які ознаки є числовими, а які категоріальні, якщо потрібно то змінити тип стовпчиків
* перевірити дані на наявність пропущених значень, зробити висновки які дані залишити, а які видалити
* перевірити дані на наявність викидів, при необхідності створити **нові дані** без викидів
* вивести гістограми для числових ознак, кількість стовпчиків має бути рівна квадратному кореню з кількості рядків, дати опис гістограмам:
  1. чи видно викиди
  2. скільки є піків
  3. чи зміщенна гістограма вліво\вправо
* вивести ящики з вусами для числових ознак, дати опис ящикам:
  1. в якому діапазоні зосереджено 25%\50%\75%\100% даних
  2. чи є викиди
  3. чи зміщені ящики вліво\вправо
* вивести гістограми з кількостями для категоріальних ознак
* вивести групові ящики з вусами для числової та категоріальної ознаки, вказати від якої категоріальної ознаки залежить числова ознака і як саме
* вивести точкові діаграми для числових ознак, вказати для яких числових ознак є залежність

**Побудова моделі для задач класифікації та регресії**
* при необхідності згенерувати нові ознаки
* самостійно обрати модель для тренування або обрати за допомогою `optuna`
* розділити **дані без викидів** на тренувальні та тестові, можливо валідаційні якщо потрібно
* вибрати метрики для оцінки якості моделі, вибір пояснити
* побудувати препроцесор
* натренувати модель
* вивести метрики для тестових даних, дати оцінку якості моделі
* зберегти натреновану модель

**Додатково**
* порівняти результат моделі на неочищених від викидів даних
* застосувати підбір гіперпараметрів

In [13]:
import pandas as pd
# Зчитування файлу Excel
df = pd.read_excel("/content/дані.xlsx")
# Виведення перших декількох рядків для перевірки
df.head()

,case_id,"Швидкість притиру, мм/сек","Напруга живлення притиру, В:","Частота обертання шліфувального круга, об/хв","Перетворювач частоти ДАНФОСС VLT MICRO DRIVE FC51, Гц",матеріал притиру,присутній на дослідженнях,дні експерименту,"Потужність притиру, кВт",Ефективність (%),"Витрати енергії, kWh","Температура, °C","Вимірювання шуму, дБ",Модель станка,Кут нахилу або нахил шліфувального станка:
0,1.0,0.10,0.08-0.3,680.0,15.0,мінералокераміка,лаборант,перший,0.0015,83.0,0.03,25.0,65.0,XYZ-500,7.0
1,2.0,0.40,0.07-0.4,975.0,23.0,мінералокераміка,аспірант,другий,0.0096,22.0,0.05,30.0,70.0,ABG-2000,8.0
2,3.0,0.05,0.1-0.7,570.0,37.0,металокераміка,професор,третій,0.0228,57.0,0.07,28.0,68.0,CNC-8000,6.0
3,4.0,0.60,0.57 - 0.63,1980.0,47.0,мінералокераміка,аспірант,четвертий,0.1782,95.0,0.04,32.0,72.0,EFG-450,9.0
4,5.0,0.15,0.67 - 0.73,2400.0,52.0,металокераміка,професор,п'ятий,0.0624,41.0,0.06,27.0,67.0,JKL-1200,10.0


ознайомитись з даними, визначити які ознаки є числовими, а які категоріальні, якщо потрібно то змінити тип стовпчиків

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 15 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   case_id                                                49 non-null     float64
 1   Швидкість притиру, мм/сек                              49 non-null     float64
 2   Напруга живлення притиру, В:                           49 non-null     object 
 3   Частота обертання шліфувального круга, об/хв           46 non-null     float64
 4   Перетворювач частоти ДАНФОСС VLT MICRO DRIVE FC51, Гц  50 non-null     float64
 5   матеріал притиру                                       47 non-null     object 
 6   присутній на дослідженнях                              48 non-null     object 
 7   дні експерименту                                       48 non-null     object 
 8   Потужність притиру, кВт                             

In [15]:
df.head()

,case_id,"Швидкість притиру, мм/сек","Напруга живлення притиру, В:","Частота обертання шліфувального круга, об/хв","Перетворювач частоти ДАНФОСС VLT MICRO DRIVE FC51, Гц",матеріал притиру,присутній на дослідженнях,дні експерименту,"Потужність притиру, кВт",Ефективність (%),"Витрати енергії, kWh","Температура, °C","Вимірювання шуму, дБ",Модель станка,Кут нахилу або нахил шліфувального станка:
0,1.0,0.10,0.08-0.3,680.0,15.0,мінералокераміка,лаборант,перший,0.0015,83.0,0.03,25.0,65.0,XYZ-500,7.0
1,2.0,0.40,0.07-0.4,975.0,23.0,мінералокераміка,аспірант,другий,0.0096,22.0,0.05,30.0,70.0,ABG-2000,8.0
2,3.0,0.05,0.1-0.7,570.0,37.0,металокераміка,професор,третій,0.0228,57.0,0.07,28.0,68.0,CNC-8000,6.0
3,4.0,0.60,0.57 - 0.63,1980.0,47.0,мінералокераміка,аспірант,четвертий,0.1782,95.0,0.04,32.0,72.0,EFG-450,9.0
4,5.0,0.15,0.67 - 0.73,2400.0,52.0,металокераміка,професор,п'ятий,0.0624,41.0,0.06,27.0,67.0,JKL-1200,10.0


перевірити дані на наявність пропущених значень, зробити висновки які дані залишити, а які видалити

In [16]:
df=df.dropna()

In [17]:
df= df.drop(columns=['матеріал притиру'])

In [19]:
df= df.drop(columns=['присутній на дослідженнях'])

In [20]:
df= df.drop(columns=['Потужність притиру, кВт'])

In [21]:
df= df.drop(columns=['Модель станка'])

In [23]:
df= df.drop(columns=['дні експерименту'])

In [24]:
df.head()

,case_id,"Швидкість притиру, мм/сек","Напруга живлення притиру, В:","Частота обертання шліфувального круга, об/хв","Перетворювач частоти ДАНФОСС VLT MICRO DRIVE FC51, Гц",Ефективність (%),"Витрати енергії, kWh","Температура, °C","Вимірювання шуму, дБ",Кут нахилу або нахил шліфувального станка:
0,1.0,0.10,0.08-0.3,680.0,15.0,83.0,0.03,25.0,65.0,7.0
1,2.0,0.40,0.07-0.4,975.0,23.0,22.0,0.05,30.0,70.0,8.0
2,3.0,0.05,0.1-0.7,570.0,37.0,57.0,0.07,28.0,68.0,6.0
3,4.0,0.60,0.57 - 0.63,1980.0,47.0,95.0,0.04,32.0,72.0,9.0
4,5.0,0.15,0.67 - 0.73,2400.0,52.0,41.0,0.06,27.0,67.0,10.0


In [30]:
# Функція для заміни значення діапазону на середнє
def replace_range_with_average(value):
    if isinstance(value, str) and '-' in value:
        # Розбиття значення на числа
        start, end = map(float, value.split('-'))
        # Розрахунок середнього значення
        return (start + end) / 2
    else:
        # Повернення вхідного значення, якщо воно не є діапазоном
        return value

# Заміна значень діапазону на середнє
df['Напруга живлення притиру, В:'] = df['Напруга живлення притиру, В:'].apply(replace_range_with_average)

In [31]:
df.head()

,case_id,"Швидкість притиру, мм/сек","Напруга живлення притиру, В:","Частота обертання шліфувального круга, об/хв","Перетворювач частоти ДАНФОСС VLT MICRO DRIVE FC51, Гц",Ефективність (%),"Витрати енергії, kWh","Температура, °C","Вимірювання шуму, дБ",Кут нахилу або нахил шліфувального станка:
0,1.0,0.10,0.190,680.0,15.0,83.0,0.03,25.0,65.0,7.0
1,2.0,0.40,0.235,975.0,23.0,22.0,0.05,30.0,70.0,8.0
2,3.0,0.05,0.400,570.0,37.0,57.0,0.07,28.0,68.0,6.0
3,4.0,0.60,0.600,1980.0,47.0,95.0,0.04,32.0,72.0,9.0
4,5.0,0.15,0.700,2400.0,52.0,41.0,0.06,27.0,67.0,10.0


перевірити дані на наявність викидів, при необхідності створити нові дані без викидів